# Loading the data

In [2]:
from keras.datasets import boston_housing
(training_data, training_targets), (test_data, test_targets) = boston_housing.load_data()

In [4]:
# take a look at the data
print(training_data.shape)
print(training_labels.shape)

print(test_data.shape)

(404, 13)
(404,)
(102, 13)


# Preprocessing the data